# Analysis of user story points changes

We aim to gain a better understanding of the properties of user stories regarding the size of the task. Therefore, we start off with formulating a hypothesis regarding changes made to the size of a task after awarding it initially with a number of story points (which may be based on a t-shirt sizing, in practice). Using this hypothesis, we hope to understand more about why story points change and how we could use this to improve our estimation of work.

The goal is to find a way to improve our current backlog estimation methods by introducing a new correction factor that determines how the backlog at a current moment in time may change based on potential future adjustments to the story points. This may be used along with changes due to new stories, removed stories, and stories that have not yet been awarded points in order to generate a full backlog size estimation.

**Hypothesis**: There is a "correction factor" which can be calculated from a data set of user stories. The correction factor describes the difference between a backlog based on initial awarding of points and a backlog of adjusted story points.

**Null hypothesis**: The correction factor stays 1, regardless of the changes to user story points.

In this report, we go through the ways we collect the data required for this analysis and the way we look at the statistics that we obtain from it. We also look into which statistical measures appear relevant for this analysis and how we could use them for our benefit in case we can reject the null hypothesis.

In [ ]:
# Correct location of notebook
setwd('/home/notebook/data-analysis')

In [ ]:
# Import database functions
if (exists('INC_DATABASE_R')) {
    rm(INC_DATABASE_R)
}
source('include/database.r')
# Import plotting and data manipulation libraries
library(ggplot2)
library(plyr)

# Create a mode function
mode <- function(data, na.rm=F) {
    unique_data <- unique(data[!is.na(data)])
    return(unique_data[which.max(tabulate(match(data, unique_data)))])
}

First, we want to know how often story points are changed. This helps us understand the scope of the data compared to the entire data set.

In [ ]:
# Create database connection
conn <- connect()
# Retrieve patterns, such as what a story point is
patterns <- load_definitions('sprint_definitions.yml')
patterns$story_points <- 'CASE WHEN ${t("issue")}.story_points >= 99 THEN 0 ELSE ${t("issue")}.story_points END'

In [ ]:
# Collect:
# - Project name (from HQ if possible)
# - JIRA issue key
# - JIRA issue ID (internal, consistent throughout board changes)
# - JIRA sprint ID
# - The first before which the change occurred (NA if there is no change)
# - The story points before the change occurred
# - The story points after the change occurred
# - The moment the sprint is really started (1 day leeway)
# - The moment at which the change occurred
# - The moment the sprint is closed
# - Whether the final version of the story indicates that it was "removed" from the backlog
# From issues that:
# - Are stories
# - Have story points
# - (Optionally) old version has story points
# If there is no change where the old version had story points, then provide the latest version
changes_query <- '
SELECT COALESCE(project.quality_display_name, project.name) AS project, project.name AS project_key,
        issue.key, issue.issue_id, issue.sprint_id, old_issue.changelog_id,
        old_issue.story_points AS old_points, issue.story_points AS new_points,
        ${s(sprint_open)} AS start, issue.updated, ${s(sprint_close)} AS close,
        ${s(planned_end)} < issue.updated AND issue.updated < ${s(sprint_close)} AS late,
        ${s(issue_excluded, issue="final_issue")} AS removed
    FROM gros.issue
    JOIN (SELECT issue_id, MAX(changelog_id) AS changelog_id
        FROM gros.issue GROUP BY issue_id
    ) AS max_issue ON issue.issue_id = max_issue.issue_id
    JOIN gros.issue AS final_issue
    ON issue.issue_id = final_issue.issue_id AND max_issue.changelog_id = final_issue.changelog_id
    LEFT JOIN gros.issue AS old_issue
    ON issue.project_id = old_issue.project_id
    AND issue.issue_id = old_issue.issue_id
    AND issue.changelog_id = old_issue.changelog_id + 1
    AND (
        COALESCE(${s(story_points)}, 0) <> COALESCE(${s(story_points, issue="old_issue")}, 0)
        --OR COALESCE(issue.approved, FALSE) <> COALESCE(old_issue.approved, FALSE)
        --OR COALESCE(issue.approved_by_po, FALSE) <> COALESCE(old_issue.approved_by_po, FALSE)
        --OR COALESCE(issue.ready_status, 0) <> COALESCE(old_issue.ready_status, 0)
    )
    JOIN gros.project ON issue.project_id = project.project_id
    LEFT JOIN gros.sprint ON issue.project_id = sprint.project_id AND issue.sprint_id = sprint.sprint_id
    WHERE COALESCE(project.is_support_team, FALSE) = FALSE
    AND ${s(issue_story)}
    AND COALESCE(${s(story_points)}, 0) > 0
    AND COALESCE(${s(story_points, issue="old_issue")}, 1) > 0
    AND (old_issue.issue_id IS NOT NULL OR issue.changelog_id = final_issue.changelog_id)
ORDER BY project.quality_display_name, project.name, issue.issue_id, issue.changelog_id'
time <- system.time(changes_data <- dbGetQuery(conn, load_query(list(query=changes_query), patterns)$query))
cat(paste("Query took", time['elapsed'], "seconds"), "\n")
cat("There are", nrow(changes_data), "versions of Jira items with story points")

In [ ]:
# The total number of Jira items with story points
length(unique(changes_data$issue_id))

In [ ]:
edit_data <- changes_data[!is.na(changes_data$changelog_id) & !is.na(changes_data$issue_id), ]
multiple_changes <- edit_data[duplicated(edit_data$issue_id), ]
# Number of Jira items with multiple changes
length(unique(multiple_changes$issue_id))
nrow(multiple_changes)

In [ ]:
# The number of Jira items that end up being removed from the backlog
length(unique(changes_data[changes_data$removed, "issue_id"]))

In [ ]:
# The number of Jira items that has not been in a sprint
in_sprint <- unique(changes_data[!is.na(changes_data$sprint_id), "issue_id"])
length(unique(changes_data[!(changes_data$issue_id %in% in_sprint), "issue_id"]))

In [ ]:
# The number of Jira items that have their points changed during a sprint
late <- changes_data[changes_data$late & !is.na(changes_data$changelog_id), c("project", "key", "changelog_id", "start", "updated", "close", "sprint_id", "old_points", "new_points")]
length(unique(late$key))
late[late$updated > Sys.Date() - as.difftime(12, units="weeks"),]

A large number of stories has at least one change to their story points during the sprint on which they are supposed to be worked on. This can indicate any of the following:

- The task is found to be larger than initially expected and the story points are therefore adjusted. Since the story points should be an indicator of the expected number of points, this is an incorrect usage of the field.
- The task was partially worked on during the sprint and the story points are adjusted to show the remaining number of points. This would assume that the tasks could have been split up and it is therefore also wrong to adjust the original points to only consider the remaining points.
- Planning is adjusted late when the sprint should be underway. It is unclear if the commitment to work on a number of points could actually be met.
- An error in estimation is corrected.

We now look into whether there is a difference in statistics (such as central tendency measurements, sizes and outliers) in each step in which a story has its story points changed. We look at all the changes per step and calculate these statistics.

In [ ]:
#TODO: Onderzoek naar andere status-indicatoren en juist deze meetellen voor onderzoek. Dus als story in sprint wordt
#genomen, dan alle versies van punten tot dan toe meenemen.

# All story changes that:
# - actually are changes (not a single story point addition)
# - have been in a sprint at any moment
# - do not have their story points or other indicators changed during a sprint
# - did not result in a "removed" story

total <- setdiff(unique(changes_data$issue_id), unique(changes_data[!is.na(changes_data$old_points), "issue_id"]))
#length(unique(changes_data[!(changes_data$issue_id %in% in_sprint), "issue_id"]))
#length(unique(changes_data[changes_data$late, "issue_id"]))
#length(unique(changes_data[changes_data$removed, "issue_id"]))
#length(unique(good_changes$issue_id))

good_changes <- changes_data[!is.na(changes_data$old_points) & changes_data$issue_id %in% in_sprint &
                             !changes_data$removed,]
length(unique(good_changes$issue_id)) / length(total) * 100

We are therefore looking at about 12% of the stories in our data set that have timely changes to their story points. All others do not have story points, were never changed, or were removed. This seems like a fair amount. The main part of the backlog consists of stories whose points do not need changing after they have been refined. Others may be changed and this could have an impact on planning.

In [ ]:
steps_data <- do.call("rbind", lapply(split(good_changes, good_changes$issue_id), function(issue) {
    issue$step <- 1:nrow(issue)
    return(issue)
}))

In [ ]:
calc <- function(df) {
    return(c(paste("Average change:", mean(df$new_points - df$old_points)),
             paste("Median change:", median(df$new_points - df$old_points)),
             paste("Mode of change:", mode(df$new_points - df$old_points))))
}
sign_name <- function(df) {
    signs <- list("-1"="negative", "0"="neutral", "1"="positive")
    return(signs[as.character(sign(df[1, "new_points"] - df[1, "old_points"]))])
}

for (step in split(steps_data, steps_data$step)) {
    cat(paste(c(paste("step ", step[1, "step"] - 1, " -> ", step[1, "step"], " (", nrow(step), " instances):", sep=""),
                calc(step), ""), collapse="\n"))
    for (sgn in split(step, sign(step$new_points - step$old_points))) {
        cat(paste(c(paste(sign_name(sgn), " (", nrow(sgn), " instances):", sep=""),
                    calc(sgn), ""), collapse="\n"))
    }
    cat("\n")
}

In [ ]:
first_to_last <- do.call("rbind", lapply(split(steps_data, steps_data$issue_id), function(issue) {
    return(data.frame(issue_id=issue[1, "issue_id"], old_points=issue[1, "old_points"], new_points=issue[nrow(issue), "new_points"]))
}))
cat(paste(c("first -> last:",
            calc(first_to_last), ""), collapse="\n"))
for (sgn in split(first_to_last, sign(first_to_last$new_points - first_to_last$old_points))) {
    cat(paste(c(paste(sign_name(sgn), " (", nrow(sgn), " instances):", sep=""),
                calc(sgn), ""), collapse="\n"))
}


We see that in each step the changes in points differ in their averages. Given that a large majority of stories only have one change, the difference in the first step `#0 -> #1` is similar to the overall step `first -> last`.

There are usually more changes due to which the number of points decreases (negative trend) compared to an increase of points. Usuaully the decrease is stronger than the increase in the same step, leading to an overall decrease of the points in that data set.

Note that the median or mode provide a more realistic statistic than the average, both in terms of the use of fixed numbers for story point changes as well as when comparing the direction of the overall change in the data set. The average is too biased due to large outliers in changes, making it hard to compare. See for example `step #2 -> #3` where the median and more are much less biased due to a small number of stories that have a large decrease in points.

Only a small subset of stories eventually revert back to their original points.

In [ ]:
steps_stats <- do.call("rbind", lapply(split(steps_data, steps_data$step), function(step) {
    diff <- step$new_points - step$old_points
    quant <- quantile(diff)
    IQR <- quant["75%"] - quant["25%"]
    return(data.frame(step=as.character(step[1, "step"]),
                      min=min(diff),
                      max=max(diff),
                      lower=quant["25%"],
                      middle=quant["50%"],
                      upper=quant["75%"],
                      ymin=min(diff[diff >= quant["25%"] - 1.5*IQR]),
                      ymax=max(diff[diff <= quant["75%"] + 1.5*IQR])))
}))
steps_stats

In [ ]:
ggplot(steps_data, aes(x=as.character(step), y=new_points - old_points, group=step)) +
    geom_boxplot() +
    scale_x_discrete("Step") +
    scale_y_continuous("Range of Changes", limits=c(-10, 10)) # Remove a large portion of outliers for now
# TODO: Display the results from splits into positive/negative as well, and the "total". Think about Sankey as well.

Statisically speaking, the differences between the groups do not seem relevant enough to consider them to be largely different, aside from a few specific steps that show some difference in their central tendency statistic which seems robust. This is also the case for the group of stories from the first change to their story points until their last change, which is hugely dependent on the group for the first step. The last three steps are also too small to realistically consider for a comparative analysis.

This requires a little more statistical testing to verify.

This may mean that we can calculate a correction factor for the entire data set instead of for each change specifically, or that we can calculate a correction factor for groups of stories based on different properties. The way we would calculate a correction factor for a data set (such as a backlog) could then be: `mode of story point changes / total number of stories`. (Alternatively: `(median of negative changes + median of positive changes) / total number of stories`)

Further research could look into stories that have other properties, such as ready status, relative time of the story point change (compared to the start of the sprint, milestones in the project or meetings related to planning and refinement), links to other issues (relationships, epics) and textual fields. The hypothesis could then be, for example: A different correction factor may be calculated for stories that have more details filled in, compared to those that have fewer details.

In [ ]:
# Actually calculate a (sample) correction factor, taking other stories into account
old_correction_factor <- function(changes, total) {
    1 + mode(changes$new_points - changes$old_points) / length(unique(total[!is.na(changes_data$old_points), "issue_id"]))
}
correction_factor <- function(changes, total) {
    neg <- changes[changes$new_points - changes$old_points < 0,]
    pos <- changes[changes$new_points - changes$old_points >= 0,]
    1 + (pmax(-1, median(pos$new_points - pos$old_points) + median(neg$new_points - neg$old_points), na.rm=T)) / length(unique(total[!is.na(changes_data$old_points), "issue_id"]))
}

cat(sprintf("Overall correction factor (all stories, all changes) is %.2f\n",
          correction_factor(steps_data, changes_data)))
corrs <- lapply(split(steps_data, steps_data$project), function(project) {
    project_name <- project[1, "project"]
    all_stories <- changes_data[changes_data$project == project_name,]
    corr <- correction_factor(project, all_stories)
    old_corr <- old_correction_factor(project, all_stories)
    cat(sprintf("Correction factor for %s is %.2f (%.2f)\n", project_name, corr, old_corr))
    num_multi <- length(unique(multiple_changes[multiple_changes$project == project_name, "issue_id"]))
    return(c(project_name, corr, length(unique(all_stories$issue_id)), num_multi))
})
nums <- c("correction_factor", "num_stories", "num_multi")
corrs <- t(as.data.frame(corrs))
colnames(corrs) <- c("project_name", nums)
corrs <- as.data.frame(corrs)
corrs[nums] <- lapply(corrs[nums], function(f) { as.numeric(levels(f))[f] })

# plot correctiefactor en # stories
corrs

In [ ]:
# TODO: Perform validation using an initial backlog
# and see how much the correction factor deviates from the actual value of a later backlog
# TODO: Count teams ("boards") together instead of in separate projects

changes_data[changes_data$project == "<project>" & !is.na(changes_data$old_points),
             c("key", "changelog_id", "old_points", "new_points")]

In [ ]:
ggplot(corrs, aes(x=correction_factor, y=num_stories)) +
    geom_point(aes(size=num_multi)) +
    geom_text(aes(label=project_name), hjust=0.5, vjust=-1.75, size=3) +
    stat_smooth(formula=y ~ x, method="loess", se=F, show.legend=F) +
    scale_x_continuous("Correction factor", breaks=seq(0.5, 1.3, 0.1)) +
    scale_y_continuous("Number of stories") +
    scale_size(name="#stories > 1 change", guide="legend") +
    labs(title=paste("Correction factor per project")) +
    theme(axis.text.x = element_text(angle=90, hjust=1, vjust=0.5), plot.title=element_text(hjust=0.5))


In [ ]:
ggplot(corrs, aes(y=num_stories, x=num_multi / num_stories)) +
    geom_point(aes(size=correction_factor)) +
    geom_text(aes(label=project_name), nudge_x=0, nudge_y=50, size=3) +
    stat_smooth(formula=y ~ x, method="loess", se=F, show.legend=F) +
    scale_y_continuous("Number of stories") +
    scale_x_continuous("% of stories > 1 change") +
    scale_size(name="Correction factor", limits = c(0.5, 1.3), guide="legend") +
    labs(title=paste("Projects with multiple story point changes")) +
    theme(axis.text.x = element_text(angle = 90, hjust = 1, vjust=0.5), plot.title=element_text(hjust=0.5))
